# Spark SQL


Agenda
-------------
1. Generacion de Vistas temporales
1. Realizando consultas SQL
1. Creación Base de Datos y Escritura como Tabla

- En Pyspark puedes interactuar con los datos a partir de comandos `SQL` o `DataFrame`
- `SQL` es un lenguaje de programación fácil de comprender
- Las operaciones utilizadas mediante SQL tambien pueden ser realizadas a partir de metodos de DataFrame

In [ ]:
path = '/FileStore/2015_summary'
options = {
  'header': True,
  'sep': ',',
  'inferSchema': True
}

df = spark.read.format('csv').options(**options).load(path)
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



<h3> Creación de la Vista Temporal (Contexto de Base de Datos)</h3>

In [ ]:
# Generando vista: El parametro define el nombre de la tabla para la vista
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.createOrReplaceTempView.html
df.createOrReplaceTempView('flights') # nombre de tabla

In [ ]:
# **Databricks permite usar comandos mágicos para realizar operaciones**
# Usamos %sql para realizar consultas sql
%sql
-- Realizando consultas SQL

select *
from flights
limit 10

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


**Cuando no estemos usando Databricks** Realizaremos consultas de la siguiente forma

In [ ]:
query = "Select * from flights limit 10"
df_new = spark.sql(query)

In [ ]:
df_new.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [ ]:
#  Se parece mas a un MySQL
query = """
select  *
from flights
where DEST_COUNTRY_NAME like '%{country}%'
order by count desc
""".format(country='United')

flightFilteredDf = spark.sql(query)
flightFilteredDf.show()

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|    United States|             Mexico|  7187|
|   United Kingdom|      United States|  2025|
|    United States|     United Kingdom|  1970|
|    United States|              Japan|  1496|
|    United States| Dominican Republic|  1420|
|    United States|            Germany|  1336|
|    United States|        The Bahamas|   986|
|    United States|             France|   952|
|    United States|              China|   920|
|    United States|           Colombia|   867|
|    United States|        South Korea|   827|
|    United States|            Jamaica|   712|
|    United States|        Netherlands|   660|
|    United States|             Brazil|   619|
|    United States|         Costa Rica|   608|
|    United States|        El Salvador|   508|
|    United S

## 3. Creación Tablas

Spark nos permite trabajar con bases de datos y tablas como si se tratase de una base sql tradicional

In [ ]:
def createDatabase(databaseName:str) -> str:
  """
  Crea una nueva Base de Datos
  """
  spark.sql("CREATE DATABASE IF NOT EXISTS {}".format(databaseName))
  spark.sql("USE {}".format(databaseName))
  return databaseName


In [ ]:
createDatabase('clase')

Out[10]: 'clase'

<h3>Guardando DataFrame como Tabla (Persistente) </h3>

In [ ]:
df.write.mode('overwrite').saveAsTable('flights_table')

In [ ]:
spark.sql('show tables').show()

+--------+-------------+-----------+
|database|    tableName|isTemporary|
+--------+-------------+-----------+
|   clase|flights_table|      false|
+--------+-------------+-----------+



In [ ]:
spark.sql('DROP TABLE IF EXISTS flights')

Out[13]: DataFrame[]

In [ ]:
%sql

select *
from flights
where DEST_COUNTRY_NAME like 'United%'
order by count desc


DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,United States,370002
United States,Canada,8483
United States,Mexico,7187
United Kingdom,United States,2025
United States,United Kingdom,1970
United States,Japan,1496
United States,Dominican Republic,1420
United States,Germany,1336
United States,The Bahamas,986
United States,France,952


# LABORATORIO SQL

El fichero **titanic.csv** contiene información sobre los pasajeros del Titanic. 

Escribir un programa con los siguientes requisitos:

1. Generar un DataFrame con los datos del fichero.
2. Mostrar por pantalla las dimensiones del DataFrame, el número de datos que contiene, los nombres de sus columnas y filas, los tipos de datos de las columnas, las primeras filas del DataFrame
3. Mostrar por pantalla los datos del pasajero con identificador 148.
5. Mostrar por pantalla los nombres de las personas que iban en primera clase ordenadas alfabéticamente.
6. Mostrar por pantalla el porcentaje de personas que sobrevivieron y murieron.
7. Mostrar por pantalla el porcentaje de personas que sobrevivieron en cada clase.
8. Eliminar del DataFrame los pasajeros con edad desconocida.
9. Mostrar por pantalla la edad media de las mujeres que viajaban en cada clase.
10. Añadir una nueva columna booleana para ver si el pasajero era menor de edad o no.
11. Mostrar por pantalla el porcentaje de menores y mayores de edad que sobrevivieron en cada clase.

In [ ]:
path = '/FileStore/titanic.csv'

options = {
  'header': True,
  'sep': ',',
  'inferSchema':True,
  'quote': '"'
}

df = spark.read.format('csv').options(**options).load(path)
# df = spark.read.format('csv',load=path,header=True,inferSchema=True)
df.show(3, False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |null |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |null |S       |
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+--

In [ ]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
df.count()

Out[18]: 891

In [ ]:
df.createOrReplaceTempView('titanic')

In [ ]:
%sql
-- 3. Mostrar por pantalla los datos del pasajero con identificador 148.

select *
from titanic
where PassengerId = 148



PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
148,0,3,"""Ford, Miss. Robina Maggie """"Ruby""""""",female,9.0,2,2,W./C. 6608,34.375,null,S


In [ ]:
%sql

-- 4. Mostrar por pantalla los nombres de las personas que iban en primera clase ordenadas alfabéticamente.


select name
from titanic
where Pclass = 1
order by name asc



name
"""Barber, Miss. Ellen """"Nellie"""""""
"""Bradley, Mr. George (""""George Arthur Brayton"""")"""
"""Duff Gordon, Lady. (Lucille Christiana Sutherland) (""""Mrs Morgan"""")"""
"""Duff Gordon, Sir. Cosmo Edmund (""""Mr Morgan"""")"""
"""Flynn, Mr. John Irwin (""""Irving"""")"""
"""Homer, Mr. Harry (""""Mr E Haven"""")"""
"""Mayne, Mlle. Berthe Antonine (""""Mrs de Villiers"""")"""
"""Romaine, Mr. Charles Hallace (""""Mr C Rolmane"""")"""
"""Ryerson, Miss. Susan Parker """"Suzette"""""""
"Allen, Miss. Elisabeth Walton"


In [ ]:
%sql
-- 5. Mostrar por pantalla el porcentaje de personas que sobrevivieron y murieron.


select Survived, (count(*) / 891 * 100)
from titanic
group by Survived



Survived,((CAST(count(1) AS DOUBLE) / CAST(891 AS DOUBLE)) * CAST(100 AS DOUBLE))
1,38.38383838383838
0,61.61616161616161


# RELACIONADO
----------------------

- Capitulo 10: Spark SQL - Spark The Define Guide
- [Spark SQL Reference](https://spark.apache.org/docs/3.0.0-preview/sql-ref-null-semantics.html)